In [40]:
#files to open: train and test data,

from google.colab import files

uploaded = files.upload()


Saving test_y1_v8.csv to test_y1_v8.csv
Saving test_v4_7615.csv to test_v4_7615.csv
Saving test_v3.csv to test_v3.csv
Saving test_v7_v8.csv to test_v7_v8.csv


# Imports and functions

In [0]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.utils import class_weight
from sklearn.metrics import recall_score, precision_score, confusion_matrix

In [3]:
import numpy as np
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation

Using TensorFlow backend.


In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

def plot_loss(epochs, loss, val_loss):
    plt.figure(figsize=(10, 7))
    plt.plot(np.arange(epochs), loss, label='loss')
    plt.plot(np.arange(epochs), val_loss, label='val_loss')
    plt.legend()
    plt.grid(True)
def acc_graph(epochs, train_acc, val_acc):
    x_axis = np.linspace(1, epochs, num=epochs)
    
    plt.figure(figsize=(10, 7))
    plt.plot(x_axis, train_acc, color='g', lw=3, alpha=0.7, label='Train Accuracy')
    plt.plot(x_axis, val_acc, color='orange', lw=3, alpha=0.7, label='Val Accuracy')
    plt.title('Accuracy graph')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')
    plt.grid(True)

In [0]:
def calculation_classes(y, y_test = None):
    print ('\t\tTRAIN DATA\tTEST DATA')
    print ('Negative\t', np.sum(y==0),'  ',
           round(100*np.sum(y==0)/y.shape[0],2),'%',
           '\t',np.sum(y_test==0),'  ',
           round(100*np.sum(y_test==0)/y_test.shape[0],2),'%'
           '\nPositive\t ', np.sum(y==1),' ',
           round(100*np.sum(y==1)/y.shape[0],2),'%'
           '\t',np.sum(y_test==1),' ',
           round(100*np.sum(y_test==1)/y_test.shape[0],2),'%'
      )

In [0]:
#from utils import model

In [0]:
def scale_0_1_for_samples(X):
  mm = MinMaxScaler()

  for i in range(X.shape[0]):
    line = X[i]
    #print(line)
  
    line = line.reshape(-1, 1)
    X[i] = mm.fit_transform(line).reshape(1,-1)
  return X
 
def scale_standard_for_1sample(X):
  ss = StandardScaler()

  for i in range(X.shape[0]):
    line = X[i]
    X[i] = ss.fit_transform(line.reshape(-1,1)).reshape(1,-1)
  return X

def scale_substraction_mean_for_1sample(X):
  for i in range(X.shape[0]):
    line = X[i]
    #print(line)
    mean = np.mean(line)
    #print(mean)
    X[i] = line - mean 
    #if min(X[i]) < 0:
    #  X[i] -= min(X[i])*1.2
    #print(X[i])
    
  return X
    

In [0]:
def get_mean_for_features(X):
  m = X.shape[0]
  X_mean = np.zeros((m,1))
  for i in range(m):
    X_mean[i] = np.mean(X[i,:], axis=0)
  return X_mean

def scale_standard_for_samples(X, X_mean):
  ss = StandardScaler()
  ss.fit(X_mean)

  X = ss.transform(X)
  return X

In [0]:
def make_predictions(X, y, model, X_ini,out=False,):
  preds = model.predict(X)
  y_predict = np.where(preds > 0.5, 1, 0)
  recall = recall_score(y, y_predict)
  print('Recall = ', recall, 'Precision = ', precision_score(y, y_predict))
  cm  = confusion_matrix(y, y_predict)
  print(cm)
  if out is True:
    for i in range(X.shape[0]):
      if abs(y_predict[i] - y[i])!= 0:
        print(y[i], preds[i], X_ini[i], np.ndarray.tolist(X[i]))
    

# Import data sets

In [0]:
#data_train = pd.read_csv('sources/train.csv')
#data_test = pd.read_csv('sources/test_v7_v8.csv')

data1_train = pd.read_csv('train_y1_v8.csv')
data0_train = pd.read_csv('train_y0_v8.csv')
data0_train_rand = pd.read_csv('train_y0_rand_v8.csv')


X = np.vstack((data1_train.drop(['Y'], axis='columns').values, data0_train.drop(['Y'], axis='columns').values))
X = np.vstack((X, data0_train_rand.drop(['Y'], axis='columns').values))


y = np.hstack((data1_train['Y'].values, data0_train['Y'].values))
y = np.hstack((y, data0_train_rand['Y'].values))

X_ini = X.copy()

In [0]:
data_add = pd.read_csv('train_0_change_1_to_close.csv')

X_add = data_add.drop(['Y'], axis='columns').values
y_add = data_add['Y'].values

X = np.vstack((X, X_add))
y = np.hstack((y, y_add))

In [0]:
data_v4 = pd.read_csv('train_0_v4.csv')

X_add = data_add.drop(['Y'], axis='columns').values
y_add = data_add['Y'].values

X = np.vstack((X, X_add))
y = np.hstack((y, y_add))

In [0]:
scale_X = scale_substraction_mean_for_1sample

In [158]:
X = scale_X(X)

X_train,X_dev,y_train,y_dev = train_test_split(X, y, test_size=0.2, \
                                               shuffle=True, stratify=y)

class_weights_train = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y), y)
print(class_weights)
print(class_weights[0]/(class_weights[1]+class_weights[0]))
calculation_classes(y, y_dev)

X = np.expand_dims(X, axis=2)
X_train = np.expand_dims(X_train, axis=2)
X_dev = np.expand_dims(X_dev, axis=2)

[0.57129108 4.00674995]
0.12478941942621506
		TRAIN DATA	TEST DATA
Negative	 69616    87.52 % 	 13924    87.52 %
Positive	  9926   12.48 %	 1985   12.48 %


In [159]:
print(X[300:400])

[[[-1.33992500e-01]
  [-8.54025000e-02]
  [-3.22225000e-02]
  [ 2.12095000e-02]
  [ 8.25225000e-02]
  [ 1.47885500e-01]]

 [[-1.44050667e-01]
  [-9.08706667e-02]
  [-3.74386667e-02]
  [ 2.38743333e-02]
  [ 8.92373333e-02]
  [ 1.59248333e-01]]

 [[-1.54176000e-01]
  [-1.00744000e-01]
  [-3.94310000e-02]
  [ 2.59320000e-02]
  [ 9.59430000e-02]
  [ 1.72476000e-01]]

 [[-1.68497667e-01]
  [-1.07184667e-01]
  [-4.18216667e-02]
  [ 2.81893333e-02]
  [ 1.04722333e-01]
  [ 1.84592333e-01]]

 [[-1.80784667e-01]
  [-1.15421667e-01]
  [-4.54106667e-02]
  [ 3.11223333e-02]
  [ 1.10992333e-01]
  [ 1.99502333e-01]]

 [[-1.94202333e-01]
  [-1.24191333e-01]
  [-4.76583333e-02]
  [ 3.22116667e-02]
  [ 1.20721667e-01]
  [ 2.13118667e-01]]

 [[-2.08317667e-01]
  [-1.31784667e-01]
  [-5.19146667e-02]
  [ 3.65953333e-02]
  [ 1.28992333e-01]
  [ 2.26429333e-01]]

 [[-2.21379500e-01]
  [-1.41509500e-01]
  [-5.29995000e-02]
  [ 3.93975000e-02]
  [ 1.36834500e-01]
  [ 2.39656500e-01]]

 [[-2.36491333e-01]
  [-

In [0]:
data_v4_test = pd.read_csv('test_v4_7615.csv')
X_v4 = data_v4_test.drop(['Y'], axis='columns').values
y_v4 = data_v4_test['Y'].values
test_v4 =  data_v4_test.drop(['Y'], axis='columns').values

X_v4 = scale_X(X_v4)
X_v4 = np.expand_dims(X_v4, axis=2)

In [0]:
data_v7v8 = pd.read_csv('test_v7_v8.csv')

X_v7v8 = data_v7v8.drop(['Y'], axis='columns').values
test_v7v8 =  data_v7v8.drop(['Y'], axis='columns').values
y_v7v8 = data_v7v8['Y'].values

X_v7v8 = scale_X(X_v7v8)
X_v7v8 = np.expand_dims(X_v7v8, axis=2)

In [0]:
#data_test = pd.read_csv('sources/test_v3.csv')
data_v3 = pd.read_csv('test_v3.csv')

X_v3 = data_v3.drop(['Y'], axis='columns').values
test_v3 =  data_v3.drop(['Y'], axis='columns').values
y_v3 = data_v3['Y'].values

X_v3 = scale_X(X_v3)
X_v3 = np.expand_dims(X_v3, axis=2)

# NN on train/dev sets

In [0]:
from keras.models import Model, Sequential
from keras.layers import Dense, Input, GRU, LSTM
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler


def model_GRU(input_shape, Ty, hidden_size):
    i = Input(shape=input_shape, dtype='float32')
    X = GRU(hidden_size, return_sequences=False)(i)
    
    X = Dense(Ty, activation='sigmoid')(X)
    model = Model(inputs=[i], outputs=X)
    
    return model
  
def model_LSTM(hidden_size):  
  model = Sequential()
  model.add(LSTM(hidden_size, return_sequences=False, input_shape=(Tx,1)))
  model.add(Dense(units=1, activation='sigmoid'))
  
  return model


In [91]:
GRU_model = model_LSTM(16)
GRU_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 16)                1152      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 1,169
Trainable params: 1,169
Non-trainable params: 0
_________________________________________________________________


In [0]:
# For ready model
#GRU_model = model_GRU((Tx,1),Ty)#LSTM_model.summary()
GRU_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


GRU_model.load_weights('LSTM_sequence_weights.h5')

In [0]:
Tx=  6
Ty = 1
epochs = 5
batch_size = 32

In [165]:
GRU_model = model_LSTM(16)
opt = Adam(lr=0.02, beta_1=0.9, beta_2=0.999)
GRU_model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
logs = GRU_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, shuffle=True,\
                    validation_data=(X_dev, y_dev), verbose=2, class_weight=class_weights_train, \
                    initial_epoch=0)

Train on 63633 samples, validate on 15909 samples
Epoch 1/5
 - 11s - loss: 0.1633 - acc: 0.9438 - val_loss: 0.0124 - val_acc: 0.9970
Epoch 2/5
 - 7s - loss: 0.0111 - acc: 0.9965 - val_loss: 0.0063 - val_acc: 0.9979
Epoch 3/5
 - 7s - loss: 0.0083 - acc: 0.9975 - val_loss: 0.0073 - val_acc: 0.9974
Epoch 4/5
 - 7s - loss: 0.0066 - acc: 0.9981 - val_loss: 0.0052 - val_acc: 0.9992
Epoch 5/5
 - 7s - loss: 0.0064 - acc: 0.9981 - val_loss: 0.0054 - val_acc: 0.9989


In [0]:
opt = Adam(lr=0.0003, beta_1=0.9, beta_2=0.999, decay=0.001)
GRU_model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [172]:
n_epochs = 5
logs = GRU_model.fit(X_train, y_train , epochs=n_epochs, batch_size=batch_size, shuffle=True,\
                    validation_data=(X_dev, y_dev), verbose=2, class_weight=class_weights_train)

Train on 63633 samples, validate on 15909 samples
Epoch 1/5
 - 7s - loss: 3.6560e-04 - acc: 0.9999 - val_loss: 6.8111e-04 - val_acc: 0.9997
Epoch 2/5
 - 7s - loss: 3.6228e-04 - acc: 0.9999 - val_loss: 6.7895e-04 - val_acc: 0.9997
Epoch 3/5
 - 7s - loss: 3.5831e-04 - acc: 0.9999 - val_loss: 6.7732e-04 - val_acc: 0.9997
Epoch 4/5
 - 7s - loss: 3.5654e-04 - acc: 0.9999 - val_loss: 6.7366e-04 - val_acc: 0.9997
Epoch 5/5
 - 7s - loss: 3.5244e-04 - acc: 0.9999 - val_loss: 6.7414e-04 - val_acc: 0.9997


In [173]:
make_predictions(X,y, GRU_model,X_ini, out=True)

Recall =  1.0 Precision =  0.9989935587761675
[[69606    10]
 [    0  9926]]
0.0 [0.77351594] [-0.08  -0.044 -0.01   0.023  0.07   0.106] [[-0.09083333333333334], [-0.05483333333333334], [-0.020833333333333332], [0.012166666666666668], [0.05916666666666667], [0.09516666666666666]]
0.0 [0.77351594] [-0.08  -0.044 -0.01   0.023  0.07   0.106] [[-0.09083333333333334], [-0.05483333333333334], [-0.020833333333333332], [0.012166666666666668], [0.05916666666666667], [0.09516666666666666]]
0.0 [0.54393864] [-0.08  -0.044  0.008  0.051  0.07   0.106] [[-0.0985], [-0.0625], [-0.010499999999999999], [0.0325], [0.051500000000000004], [0.0875]]
0.0 [0.6399604] [-0.032 -0.006  0.008  0.023  0.033  0.077] [[-0.049166666666666664], [-0.02316666666666667], [-0.009166666666666667], [0.005833333333333333], [0.015833333333333335], [0.059833333333333336]]
0.0 [0.6399604] [-0.032 -0.006  0.008  0.023  0.033  0.077] [[-0.049166666666666664], [-0.02316666666666667], [-0.009166666666666667], [0.005833333333333

In [0]:
#make_predictions(X_dev,y_dev, out=True, model=GRU_model)

In [174]:
make_predictions(X_v7v8, y_v7v8, out=True, model=GRU_model, X_ini=test_v7v8)

Recall =  1.0 Precision =  1.0
[[13  0]
 [ 0 46]]


In [175]:
make_predictions(X_v3, y_v3, out=True, model=GRU_model, X_ini=test_v3)

Recall =  1.0 Precision =  1.0
[[42  0]
 [ 0 53]]


In [176]:
make_predictions(X_v4, y_v4, out=True, model=GRU_model, X_ini=test_v4)

Recall =  0.0 Precision =  0.0
[[66  5]
 [ 0  0]]
0.0 [0.69436234] [-0.4   -0.334 -0.226 -0.11   0.023  0.169] [[-0.2536666666666667], [-0.18766666666666662], [-0.07966666666666666], [0.03633333333333334], [0.16933333333333334], [0.31533333333333335]]
0.0 [0.84127694] [-0.38  -0.334 -0.226 -0.11   0.023  0.169] [[-0.237], [-0.19099999999999995], [-0.08299999999999999], [0.033000000000000015], [0.166], [0.312]]
0.0 [0.6637162] [-0.38  -0.299 -0.226 -0.11   0.023  0.169] [[-0.24283333333333332], [-0.1618333333333333], [-0.08883333333333332], [0.027166666666666686], [0.16016666666666668], [0.3061666666666667]]
0.0 [0.5773528] [-0.308 -0.299 -0.226 -0.11   0.023  0.169] [[-0.18283333333333335], [-0.17383333333333334], [-0.10083333333333336], [0.015166666666666648], [0.14816666666666664], [0.29416666666666663]]
0.0 [0.93948776] [ 0.284  0.274  0.192  0.09   0.023 -0.065] [[0.15099999999999997], [0.14099999999999996], [0.059], [-0.04300000000000001], [-0.11000000000000001], [-0.198]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


In [0]:
#make_predictions(X_add, y_add, out=True, model=GRU_model, X_ini=X_ini_add)

In [0]:
#plot_loss(epochs, logplot_loss(epos.history.get('loss'), logs.history.get('val_loss'))

In [0]:
#acc_graph(epochs, logs.history.get('acc'), logs.history.get('val_acc'))

In [0]:
GRU_model.save_weights('LSTM_sequence_weights.h5')

In [0]:
files.download('LSTM_sequence_weights.h5')

In [0]:
LSTM_model = model((Tx,1),Ty)
#LSTM_model.summary()
LSTM_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
logs = LSTM_model.fit(X_train, y_train , epochs=epochs, batch_size=batch_size, shuffle=True,\
               validation_data=(X_dev, y_dev), verbose=0, class_weight=class_weights_train)

# Final fit NN on all train set

In [0]:
Tx=  6
Ty = 1
epochs = 250
batch_size = 32

In [0]:
LSTM_model = model((Tx,1),Ty)
#LSTM_model.summary()
LSTM_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
LSTM_model.fit(X, y , epochs=epochs, batch_size=batch_size, shuffle=True,\
               verbose=0, class_weight=class_weights)
LSTM_model.save_weights('sources/LSTM_sequence_weights.h5')

In [0]:
print(LSTM_model.summary())
test_preds = LSTM_model.predict(X_test)

In [0]:
for i in range(X_test.shape[0]):
    if test_preds[i] > 0.5:
        print(i)

In [0]:
LSTM_model = model((Tx,1),Ty)
LSTM_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

LSTM_model.load_weights('LSTM_sequence_weights.h5')

In [0]:
test_preds = LSTM_model.predict(X_test)

for i in range(X_test.shape[0]):
    if test_preds[i] > 0.5:
        print(i)